In [ ]:
import fastf1 as ff1
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from fastf1 import plotting
import plotly.graph_objects as go

In [ ]:
ff1.Cache.enable_cache('cache')
plotting.setup_mpl()

In [ ]:
# Function to return all the sessions of a season
# Include an enumerator for race or qualifying if it is race then its An R if its is qualifying then a Q
def get_season_sessions(year, session_type):
    schedule = ff1.get_event_schedule(year)
    sessions = []
    for _, event in schedule.iterrows():
        session = ff1.get_session(year, event['Location'], session_type)
        session.load()
        sessions.append(session)
        for session in sessions:
            session.results['Position'] = np.arange(1, len(session.results) + 1)
    return sessions

In [ ]:
import concurrent.futures

years = [2023, 2022, 2021, 2020, 2019, 2018]

def process_sessions(year, session_type):
    sessions = get_season_sessions(year, session_type)
    return sessions

# Create a ThreadPoolExecutor
executor = concurrent.futures.ThreadPoolExecutor()

# Create dictionaries to store the results for R sessions and Q sessions
r_sessions_results = {}
q_sessions_results = {}

# Submit the function calls for R and Q sessions on individual threads for each year
r_futures = {executor.submit(process_sessions, year, 'R'): year for year in years}
q_futures = {executor.submit(process_sessions, year, 'Q'): year for year in years}

# Retrieve the results for R sessions from completed threads
for future in concurrent.futures.as_completed(r_futures):
    year = r_futures[future]
    try:
        result = future.result()
        r_sessions_results[year] = result
    except Exception as e:
        print(f"Error processing R sessions for year {year}: {str(e)}")

# Retrieve the results for Q sessions from completed threads
for future in concurrent.futures.as_completed(q_futures):
    year = q_futures[future]
    try:
        result = future.result()
        q_sessions_results[year] = result
    except Exception as e:
        print(f"Error processing Q sessions for year {year}: {str(e)}")

# Access the results for R sessions and Q sessions for each year
for year in years:
    r_sessions = r_sessions_results.get(year, [])
    q_sessions = q_sessions_results.get(year, [])
    print(f"R Sessions for year {year}: {r_sessions}")
    print(f"Q Sessions for year {year}: {q_sessions}")


In [ ]:
year = 2022
r_sessions = get_season_sessions(year, 'R')
q_sessions = get_season_sessions(year, 'Q')

In [ ]:
q_sessions[0].results

In [ ]:
laps = [session.laps for session in sessions]
laps = pd.concat(laps)
laps.columns

In [ ]:
ranking_qualifiers = []

for i, laps in enumerate(laps):
    quali_laps_copy = laps.copy()
    quali_laps_copy['LapTime'] = quali_laps_copy['LapTime'].dt.total_seconds()
    fastest_laps = quali_laps_copy.groupby(['Driver', 'Team'])['LapTime'].min()
    fastest_laps = fastest_laps.reset_index()
    fastest_laps['Rank'] = fastest_laps['LapTime'].fillna(100000000).rank().astype(int)
    fastest_laps = fastest_laps.sort_values('Rank')
    ranking_qualifiers.append(fastest_laps)


In [ ]:
team_average_ranks = []
for team in fastest_laps['Team'].unique():
    team_data = fastest_laps[fastest_laps['Team'] == team]
    team_average_rank = team_data['Rank'].mean()
    team_average_ranks.append((team, team_average_rank))

team_average_ranks_df = pd.DataFrame(team_average_ranks, columns=['Team', 'Average Rank'])


In [ ]:
# Collect all unique drivers and their teams
drivers_teams = {}
for df in ranking_qualifiers:
    for _, row in df.iterrows():
        drivers_teams[row['Driver']] = row['Team']

In [ ]:
fig = go.Figure()

for driver, team in drivers_teams.items():
    ranks = []
    for i, df in enumerate(ranking_qualifiers):
        rank = df[df['Driver'] == driver]['Rank'].values
        if len(rank) > 0:
            ranks.append(rank[0])
        else:
            #ranks.append(20)
            ranks.append(np.nan)  # Driver didn't participate in this qualifier

    # Compute cumulative average
    cumulative_avg = np.cumsum(ranks) / np.arange(1, len(ranks) + 1)

    fig.add_trace(go.Scatter(x=list(range(1, len(ranking_qualifiers) + 1)), y=cumulative_avg, mode='lines+markers', name=driver, marker=dict(color=plotting.team_color(team))))

fig.update_layout(
    title="Driver Average Ranking over Qualifiers",
    xaxis_title="Qualifier Number",
    yaxis_title="Average Rank",
    yaxis=dict(autorange="reversed"), # This is for the y-axis to be in reverse order
    hovermode="x"
)

fig.show()



In [ ]:
fig = go.Figure()

# We use the teams directly from the dataframe 'fastest_laps'
for team in fastest_laps['Team'].unique():
    ranks = []
    for i, df in enumerate(ranking_qualifiers):
        team_data = df[df['Team'] == team]
        # Calculate average rank for this team in this qualifier
        avg_rank = team_data['Rank'].mean()
        ranks.append(avg_rank)

    # Compute cumulative average
    cumulative_avg = np.cumsum(ranks) / np.arange(1, len(ranks) + 1)

    fig.add_trace(go.Scatter(x=list(range(1, len(ranking_qualifiers) + 1)), y=cumulative_avg, mode='lines+markers', name=team, marker=dict(color=plotting.team_color(team))))

fig.update_layout(
    title="Team Average Ranking over Qualifiers",
    xaxis_title="Qualifier Number",
    yaxis_title="Average Rank",
    yaxis=dict(autorange="reversed"),  # This is for the y-axis to be in reverse order
    hovermode="x"
)

fig.show()
